# Imports 

In [1]:
## Essential Imports: 
import os
import numpy as np
import qp
import tables_io
from pathlib import Path 
from pzflow.examples import get_galaxy_data
import ceci

## RAIL-Specific Imports: 
import rail
from rail.creation.degradation import LSSTErrorModel, InvRedshiftIncompleteness
from rail.creation.engines.flowEngine import FlowModeler, FlowCreator, FlowPosterior
from rail.core.data import TableHandle
from rail.core.stage import RailStage
from rail.core.utilStages import ColumnMapper, TableConverter
from rail.estimation.algos.flexzboost import Inform_FZBoost, FZBoost
from rail.evaluation.evaluator import Evaluator


## Data Storage: 
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

# Model

In [2]:
def makeModel():
    #path to access the data 
    DATA_DIR = Path().resolve() / "data"
    DATA_DIR.mkdir(exist_ok=True)

    catalog_file = DATA_DIR / "base_catalog.pq"

    bands = ['u','g','r','i','z','y']
    band_dict = {band:f'mag_{band}_lsst' for band in bands}
    
    #array of galaxies w/ 7 attributes for each: redshift & ugrizy
    catalog = get_galaxy_data().rename(band_dict, axis=1) 

    #turns array into a table 
    tables_io.write(catalog, str(catalog_file.with_suffix("")), catalog_file.suffix[1:])

    catalog_file = str(catalog_file)
    flow_file = str(DATA_DIR / "trained_flow.pkl")

    #we set up the stage 
    flow_modeler_params = {
        "name": "flow_modeler",
        "input": catalog_file,
        "model": flow_file,
        "seed": 0,
        "phys_cols": {"redshift": [0, 3]},
        "phot_cols": {
            "mag_u_lsst": [17, 35],
            "mag_g_lsst": [16, 32],
            "mag_r_lsst": [15, 30],
            "mag_i_lsst": [15, 30],
            "mag_z_lsst": [14, 29],
            "mag_y_lsst": [14, 28],
        },
        "calc_colors": {"ref_column_name": "mag_i_lsst"},
    }
    flow_modeler = FlowModeler.make_stage(**flow_modeler_params)
    flow_modeler.fit_model()
    return flow_modeler.get_handle("model")

In [3]:
modelData = makeModel()

Inserting handle into data store.  input: /Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/data/base_catalog.pq, flow_modeler
Training 30 epochs 
Loss:
(0) 21.3266
(1) 6.7267
(2) 2.0761
(3) 2.6037
(4) -0.0680
(5) 0.4129
(6) 0.2506
(7) 0.1637
(8) -1.3346
(9) -1.7669
(10) -1.1823
(11) -1.6267
(12) 3402823273761818485311871060541440.0000
(13) 3402823273761818485311871060541440.0000
(14) -1.0711
(15) -0.6228
(16) 3402823273761818485311871060541440.0000
(17) 3402823273761818485311871060541440.0000
(18) -2.8045
(19) -3.3746
(20) 3402823273761818485311871060541440.0000
(21) -2.4881
(22) -3.2147
(23) -3.7188
(24) -3.4398
(25) -3.7955
(26) -3.3772
(27) 3402823273761818485311871060541440.0000
(28) -3.5247
(29) -4.1677
(30) -3.4874
Inserting handle into data store.  model_flow_modeler: /Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/data/inprogress_trained_flow.pkl, flow_modeler


# Training Set 

In [9]:
def trainSet(model, ntrain, seed):
    data = FlowCreator.make_stage(
            name = 'stuff',
            model = model,
            n_samples = ntrain,
            seed = seed 
    )
    return data.sample(ntrain, seed)

def invRedshift(data, pivot = 1.0):
    deg = InvRedshiftIncompleteness.make_stage(
        name = 'stuff',
        pivot_redshift = pivot
    )
    return deg(data) 

In [10]:
origTrainData = trainSet(modelData, 10000, 372)

degTrainData = invRedshift(trainSet(modelData, 10000, 372), 1.0)

Inserting handle into data store.  output_stuff: inprogress_output_stuff.pq, stuff
Inserting handle into data store.  output_stuff: inprogress_output_stuff.pq, stuff
Inserting handle into data store.  output_stuff: inprogress_output_stuff.pq, stuff


In [13]:
def getPosts(data, model, grid):
    posts = FlowPosterior.make_stage(
        name='stuff', 
        column='redshift',
        grid = grid,
        model = model,
        data = data
    )
    return posts.get_posterior(data, column = 'redshift')

In [14]:
def makeGrid(zmin, zmax, nbins):
    import numpy as np
    grid = np.linspace(zmin, zmax, nbins + 1)
    return grid 

In [15]:
grid = makeGrid(0, 2.5, 100)

In [17]:
origTrainPosts = getPosts(origTrainData, modelData, grid)
degTrainPosts = getPosts(degTrainData, modelData, grid)

Inserting handle into data store.  output_stuff: inprogress_output_stuff.hdf5, stuff


AttributeError: 'Ensemble' object has no attribute 'reset_index'

## Posts 

Only run if you need output_orig_train_posts

In [61]:
# flow_post_orig_train = FlowPosterior.make_stage(name='orig_train_posts', 
#                                              column='redshift',
#                                              grid = np.linspace(0, 2.5, 101),
#                                              model=flow_file,
#                                              data = orig_train)

# orig_train_pdfs = flow_post_orig_train.get_posterior(orig_train, column='redshift')

Only run if you need output_deg_train_posts ** rerun this cell!! 

In [62]:
# flow_post_deg_train = FlowPosterior.make_stage(name='deg_train_posts', 
#                                              column='redshift',
#                                              grid = np.linspace(0, 2.5, 101),
#                                              model=flow_file,
#                                              err_samples = 0,
#                                              data = deg_train)



# deg_train_pdfs = flow_post_deg_train.get_posterior(deg_train, column='redshift')

# Test Set 

In [94]:
def testSet(model, ntest, seed):
    data = FlowCreator.make_stage(
            name = 'stuff',
            model = model,
            n_samples = ntest,
            seed = seed 
    )
    return data.sample(ntest, seed)


## you need to ask alex about where you can find the defaults for these params 
def lsstError(data, seed, tvis = 1, nYrObs = 1, airmass = 1, extendedSource = 1, sigmaSys = 1, magLim = 1, ndFlag = 1, A_min = 1, A_max = 1):
    bands = ['u','g','r','i','z','y']
    band_dict = {band:f'mag_{band}_lsst' for band in bands}
    deg = LSSTErrorModel.make_stage(
        name='stuff',
        bandNames=band_dict, 
        seed=seed,
    )
    return deg(data)

In [64]:
# testData = lsstError(testSet(100000, 1078), 39)

In [65]:
# testPosts = getPosts(testData, grid)

## Posts

Only run if you need output_orig_test_posts

In [66]:
# flow_post_orig_test = FlowPosterior.make_stage(name='orig_test_posts', 
#                                              column='redshift',
#                                              grid = np.linspace(0, 2.5, 101),
#                                              model=flow_file,
#                                              data = orig_test)

# orig_test_pdfs = flow_post_orig_test.get_posterior(orig_test, column='redshift')

Only run if you need output_deg_test_posts

In [67]:
# flow_post_deg_test = FlowPosterior.make_stage(name='deg_test_posts', 
#                                              column='redshift',
#                                              grid = np.linspace(0, 2.5, 101),
#                                              model=flow_file,
#                                              data = deg_test)

# deg_test_pdfs = flow_post_deg_test.get_posterior(deg_test, column='redshift')

# Make tables

In [98]:
def makeTable(datafile):
    
    bands = ['u','g','r','i','z','y']
    rename_dict = {f'mag_{band}_lsst_err':f'mag_err_{band}_lsst' for band in bands}

    col_remapper = ColumnMapper.make_stage(
    name='col_remapper', 
    columns=rename_dict,
    )
    table_conv = TableConverter.make_stage(
    name='table_conv', 
    output_format='numpyDict',
    )
    pq = col_remapper(datafile)
    tabledata = table_conv(pq)
    table = tables_io.convertObj(tabledata.data, tables_io.types.PD_DATAFRAME)
    return table


In [69]:
# trainTable = makeTable(trainData)
# testTable = makeTable(testData)

# Estimation 

In [70]:
def informFZBoost(data):
    info = Inform_FZBoost.make_stage(
    name ='informFZBoost', 
    model ='fzboost.pkl', 
    hdf5_groupname='',
    )
    info.inform(data)
    return info

In [71]:
# informedEst = informFZBoost(trainData)

In [72]:
def estimateFZBoost(data, info, nbins):
    est = FZBoost.make_stage(
    name='estFZBoost', 
    nondetect_val=np.nan,
    model= info.get_handle('model'), 
    hdf5_groupname='',
    aliases=dict(input='test_data', output='fzboost_estim'),
    nzbins = nbins ,
    zmax = zmax
    )
    return est.estimate(data)

In [84]:
# estData = estimateFZBoost(testData, informedEst, 100)

Training 30 epochs 
Loss:
(0) 21.3266
(1) 6.7267
(2) 2.0761
(3) 2.6037
(4) -0.0680
(5) 0.4129
(6) 0.2506
(7) 0.1637
(8) -1.3346
(9) -1.7669
(10) -1.1823
(11) -1.6267
(12) 3402823273761818485311871060541440.0000
(13) 3402823273761818485311871060541440.0000
(14) -1.0711
(15) -0.6228
(16) 3402823273761818485311871060541440.0000
(17) 3402823273761818485311871060541440.0000
(18) -2.8045
(19) -3.3746
(20) 3402823273761818485311871060541440.0000
(21) -2.4881
(22) -3.2147
(23) -3.7188
(24) -3.4398
(25) -3.7955
(26) -3.3772
(27) 3402823273761818485311871060541440.0000
(28) -3.5247
(29) -4.1677
(30) -3.4874
Inserting handle into data store.  model_flow_modeler: /Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/data/inprogress_trained_flow.pkl, flow_modeler


In [121]:
def bigF():
    grid = makeGrid(0, 2.5, 100) 
    #modelData = makeModel()
    trainData = invRedshift(trainSet(modelData, 10, 372), 1.0)
    trainPosts = getPosts(trainData, modelData, grid)
    testData = lsstError(testSet(modelData, 100, 1078), 39)
    testPosts = getPosts(testData, modelData, grid)
    trainTable = makeTable(trainData)
    testTable = makeTable(testData)
    informedEst = informFZBoost(trainData)
    estData = estimateFZBoost(testData, informedEst)
    pipe = ceci.Pipeline.interactive()
    stages = [modelData, trainData, trainPosts, testData, testPosts, trainTable, testTable, informedEst, estData]
    for stage in stages:
        pipe.add_stage(stage)

In [122]:
bigF()

Inserting handle into data store.  output_stuff: inprogress_output_stuff.pq, stuff
Inserting handle into data store.  output_stuff: inprogress_output_stuff.pq, stuff
Inserting handle into data store.  output_stuff: inprogress_output_stuff.hdf5, stuff
Inserting handle into data store.  output_stuff: inprogress_output_stuff.pq, stuff
Inserting handle into data store.  output_stuff: inprogress_output_stuff.pq, stuff
Inserting handle into data store.  output_stuff: inprogress_output_stuff.hdf5, stuff


/Users/alicec03/miniforge3/envs/rail/lib/python3.10/site-packages/qp/interp_pdf.py:83: RuntimeWarning: invalid value encountered in divide
  self._ycumul = (self._ycumul.T / self._ycumul[:,-1]).T


AttributeError: 'Ensemble' object has no attribute 'rename'

In [111]:
type(a)

rail.core.data.PqHandle